In [ ]:
# TorchScript
    # 跟踪现有模块
    # 使用脚本直接编译模块

In [92]:
import torch  # This is all you need to use both PyTorch and TorchScript!
print(torch.__version__)
torch.manual_seed(191009)  # set the seed for reproducibility

2.3.0+cu118


In [96]:
class MyCell1(torch.nn.Module):
    def __init__(self):
        super(MyCell1, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell1()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))
print(my_cell)
print(my_cell(x, h))

(tensor([[-0.2541,  0.2460,  0.2297,  0.1014],
        [-0.2329, -0.2911,  0.5641,  0.5015],
        [ 0.1688,  0.2252,  0.7251,  0.2530]], grad_fn=<TanhBackward0>), tensor([[-0.2541,  0.2460,  0.2297,  0.1014],
        [-0.2329, -0.2911,  0.5641,  0.5015],
        [ 0.1688,  0.2252,  0.7251,  0.2530]], grad_fn=<TanhBackward0>))
MyCell1(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[-0.2541,  0.2460,  0.2297,  0.1014],
        [-0.2329, -0.2911,  0.5641,  0.5015],
        [ 0.1688,  0.2252,  0.7251,  0.2530]], grad_fn=<TanhBackward0>), tensor([[-0.2541,  0.2460,  0.2297,  0.1014],
        [-0.2329, -0.2911,  0.5641,  0.5015],
        [ 0.1688,  0.2252,  0.7251,  0.2530]], grad_fn=<TanhBackward0>))


In [305]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x[0][0] > 0:
            return x
        else:
            return -x

class MyCell2(torch.nn.Module):
    def __init__(self):
        super(MyCell2, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h

my_cell = MyCell2()
print(my_cell)
print(my_cell(x, h))

MyCell2(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.5558, -0.5318, -0.4891,  0.4558],
        [ 0.5980, -0.0933, -0.4871,  0.8160],
        [ 0.6492, -0.2173,  0.0336,  0.7995]], grad_fn=<TanhBackward0>), tensor([[ 0.5558, -0.5318, -0.4891,  0.4558],
        [ 0.5980, -0.0933, -0.4871,  0.8160],
        [ 0.6492, -0.2173,  0.0336,  0.7995]], grad_fn=<TanhBackward0>))


In [107]:
my_cell = MyCell1()
x, h = torch.rand(3, 4), torch.rand(3, 4)
# 给定输入示例，调用了Module，记录了运行时发生的操作Module，并创建了一个实例torch.jit.ScriptModule（其中TracedModule是实例）
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
print(type(traced_cell))
traced_cell(x, h)

MyCell1(
  original_name=MyCell1
  (linear): Linear(original_name=Linear)
)
<class 'torch.jit._trace.TopLevelTracedModule'>


(tensor([[0.1917, 0.5628, 0.7345, 0.0488],
         [0.5713, 0.5418, 0.4596, 0.3537],
         [0.4559, 0.6762, 0.5744, 0.6472]], grad_fn=<TanhBackward0>),
 tensor([[0.1917, 0.5628, 0.7345, 0.0488],
         [0.5713, 0.5418, 0.4596, 0.3537],
         [0.4559, 0.6762, 0.5744, 0.6472]], grad_fn=<TanhBackward0>))

In [104]:
print(traced_cell.graph)

graph(%self.1 : __torch__.___torch_mangle_16.MyCell,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cuda:0),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cuda:0)):
  %linear : __torch__.torch.nn.modules.linear.___torch_mangle_15.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%linear, %x)
  %11 : int = prim::Constant[value=1]() # /tmp/ipykernel_68168/2516425700.py:7:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0) = aten::add(%20, %h, %11) # /tmp/ipykernel_68168/2516425700.py:7:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0) = aten::tanh(%12) # /tmp/ipykernel_68168/2516425700.py:7:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0), Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0)) = prim::TupleConstruct(%13, %13)
  return (%14)



In [108]:
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  linear = self.linear
  _0 = torch.tanh(torch.add((linear).forward(x, ), h))
  return (_0, _0)



In [106]:
# 放在cuda上也可以
my_cell = MyCell1().to("cuda")
x, h = torch.rand(3, 4), torch.rand(3, 4)
# 给定输入示例，调用了Module，记录了运行时发生的操作Module，并创建了一个实例torch.jit.ScriptModule（其中TracedModule是实例）
traced_cell = torch.jit.trace(my_cell, (x.cuda(), h.cuda()))
print(traced_cell)
print(type(traced_cell))
traced_cell(x.cuda(), h.cuda())
print(traced_cell.graph)


MyCell1(
  original_name=MyCell1
  (linear): Linear(original_name=Linear)
)
<class 'torch.jit._trace.TopLevelTracedModule'>
graph(%self.1 : __torch__.___torch_mangle_23.MyCell1,
      %x : Float(3, 4, strides=[4, 1], requires_grad=0, device=cuda:0),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cuda:0)):
  %linear : __torch__.torch.nn.modules.linear.___torch_mangle_22.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%linear, %x)
  %11 : int = prim::Constant[value=1]() # /tmp/ipykernel_68168/3666525625.py:7:0
  %12 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0) = aten::add(%20, %h, %11) # /tmp/ipykernel_68168/3666525625.py:7:0
  %13 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0) = aten::tanh(%12) # /tmp/ipykernel_68168/3666525625.py:7:0
  %14 : (Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0), Float(3, 4, strides=[4, 1], requires_grad=1, device=cuda:0)) = prim::TupleConstruct(%13, 

In [306]:
# 如果模型中包含控制流
my_cell = MyCell2()
x, h = torch.rand(3, 4), torch.rand(3, 4)
# 给定输入示例，调用了Module，记录了运行时发生的操作Module，并创建了一个实例torch.jit.ScriptModule（其中TracedModule是实例）
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
print(type(traced_cell))
traced_cell(x, h)
print(traced_cell.graph)
# 查看.code输出，我们可以看到分支if-else无处可寻！为什么？跟踪正是按照我们所说的那样：运行代码，记录发生的操作并构造一个 ScriptModule执行该操作的程序。不幸的是，控制流之类的东西被抹去了。

MyCell2(
  original_name=MyCell2
  (dg): MyDecisionGate(original_name=MyDecisionGate)
  (linear): Linear(original_name=Linear)
)
<class 'torch.jit._trace.TopLevelTracedModule'>
graph(%self.1 : __torch__.___torch_mangle_361.MyCell2,
      %x.1 : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu),
      %h : Float(3, 4, strides=[4, 1], requires_grad=0, device=cpu)):
  %dg : __torch__.___torch_mangle_359.MyDecisionGate = prim::GetAttr[name="dg"](%self.1)
  %linear : __torch__.torch.nn.modules.linear.___torch_mangle_360.Linear = prim::GetAttr[name="linear"](%self.1)
  %35 : Tensor = prim::CallMethod[name="forward"](%linear, %x.1)
  %36 : NoneType = prim::CallMethod[name="forward"](%dg, %35)
  %22 : int = prim::Constant[value=1]() # /tmp/ipykernel_68168/274459315.py:15:0
  %23 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::add(%35, %h, %22) # /tmp/ipykernel_68168/274459315.py:15:0
  %24 : Float(3, 4, strides=[4, 1], requires_grad=1, device=cpu) = aten::tanh(%23) # 

/tmp/ipykernel_68168/274459315.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x[0][0] > 0:


In [132]:
traced_cell(x,h)

(tensor([[ 0.2479,  0.6963,  0.7758, -0.1620],
         [ 0.5923,  0.4877,  0.8393,  0.2807],
         [ 0.1735,  0.4953,  0.6595, -0.4612]], grad_fn=<TanhBackward0>),
 tensor([[ 0.2479,  0.6963,  0.7758, -0.1620],
         [ 0.5923,  0.4877,  0.8393,  0.2807],
         [ 0.1735,  0.4953,  0.6595, -0.4612]], grad_fn=<TanhBackward0>))

In [133]:
# 我们提供了一个 脚本编译器，它直接分析你的 Python 源代码并将其转换为 TorchScript
class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h
    
scripted_gate = torch.jit.script(MyDecisionGate())
my_cell = MyCell(scripted_gate)
scripted_cell = torch.jit.script(my_cell)

print(scripted_gate.code)
print(scripted_cell.code)


def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.sum(x), 0)):
    _0 = x
  else:
    _0 = torch.neg(x)
  return _0

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)



In [136]:
# x, h = torch.rand(3, 4), torch.rand(3, 4)
print(scripted_cell(x, h))

(tensor([[ 0.4915,  0.0575,  0.4354,  0.5988],
        [ 0.8869,  0.2138,  0.1995,  0.7857],
        [ 0.4822, -0.1772,  0.0927,  0.1350]],
       grad_fn=<DifferentiableGraphBackward>), tensor([[ 0.4915,  0.0575,  0.4354,  0.5988],
        [ 0.8869,  0.2138,  0.1995,  0.7857],
        [ 0.4822, -0.1772,  0.0927,  0.1350]],
       grad_fn=<DifferentiableGraphBackward>))


In [220]:
print(my_cell)

MyCell2(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)


In [338]:
my_cell = MyCell2()
x, h = torch.rand(3, 4), torch.rand(3, 4)
scripted_cell = torch.jit.script(my_cell)
scripted_cell2 = torch.jit.trace(my_cell,(x,h))
print(scripted_cell(x,h))
print(scripted_cell2(x,h)[0]==scripted_cell(x,h)[0])
print(scripted_cell.code)

(tensor([[ 0.5907,  0.0780,  0.4297, -0.4303],
        [ 0.6976, -0.5335,  0.4454,  0.0253],
        [ 0.4735,  0.0993,  0.7769,  0.1649]],
       grad_fn=<DifferentiableGraphBackward>), tensor([[ 0.5907,  0.0780,  0.4297, -0.4303],
        [ 0.6976, -0.5335,  0.4454,  0.0253],
        [ 0.4735,  0.0993,  0.7769,  0.1649]],
       grad_fn=<DifferentiableGraphBackward>))
tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  dg = self.dg
  linear = self.linear
  _0 = torch.add((dg).forward((linear).forward(x, ), ), h)
  new_h = torch.tanh(_0)
  return (new_h, new_h)



In [ ]:
import torch
import torch.nn as nn

class MyCell3(nn.Module):
    def forward(self, x, h):
        if x[0][0] > 0:  # 控制流
            return x + h
        else:
            return x - h

my_cell = MyCell3()
x, h = torch.rand(3, 4), torch.rand(3, 4)
print(x.sum())
# 使用 script
scripted_cell = torch.jit.script(my_cell)

# 使用 trace
scripted_cell2 = torch.jit.trace(my_cell, (x, h))
x, h = torch.rand(3, 4), torch.rand(3, 4)
x[0][0]=1
print(scripted_cell(x,h)==scripted_cell2(x,h))
x[0][0]=-1
print(scripted_cell(x,h)==scripted_cell2(x,h))

tensor(6.5033)
tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])
tensor([[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])


/tmp/ipykernel_68168/1645319762.py:6: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum()>5:  # 控制流


In [341]:
# 模型的保存，顺便用于下一节的c++读入
my_cell = MyCell1()
x,h = torch.rand(3,4),torch.rand(3,4)
traced_cell = torch.jit.trace(my_cell, (x,h))
traced_cell.save('wrapped_rnn.pt')

loaded = torch.jit.load('wrapped_rnn.pt')

print(loaded)
print(loaded.code)

RecursiveScriptModule(
  original_name=MyCell1
  (linear): RecursiveScriptModule(original_name=Linear)
)
def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  linear = self.linear
  _0 = torch.tanh(torch.add((linear).forward(x, ), h))
  return (_0, _0)

